In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [171]:
netflix = pd.read_csv('netflix_titles.csv')
netflix.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,TV Show,3%,NaN,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,"August 14, 2020",2020,TV-MA,4 Seasons,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...
1,s2,Movie,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",Mexico,"December 23, 2016",2016,TV-MA,93 min,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...
2,s3,Movie,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",Singapore,"December 20, 2018",2011,R,78 min,"Horror Movies, International Movies","When an army recruit is found dead, his fellow..."
3,s4,Movie,9,Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly...",United States,"November 16, 2017",2009,PG-13,80 min,"Action & Adventure, Independent Movies, Sci-Fi...","In a postapocalyptic world, rag-doll robots hi..."
4,s5,Movie,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",United States,"January 1, 2020",2008,PG-13,123 min,Dramas,A brilliant group of students become card-coun...


In [172]:
netflix.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7787 entries, 0 to 7786
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       7787 non-null   object
 1   type          7787 non-null   object
 2   title         7787 non-null   object
 3   director      5398 non-null   object
 4   cast          7069 non-null   object
 5   country       7280 non-null   object
 6   date_added    7777 non-null   object
 7   release_year  7787 non-null   int64 
 8   rating        7780 non-null   object
 9   duration      7787 non-null   object
 10  listed_in     7787 non-null   object
 11  description   7787 non-null   object
dtypes: int64(1), object(11)
memory usage: 730.2+ KB


In [173]:
netflix['country'].value_counts()[:10]

United States     2555
India              923
United Kingdom     397
Japan              226
South Korea        183
Canada             177
Spain              134
France             115
Egypt              101
Mexico             100
Name: country, dtype: int64

In [201]:
feature_cols = ['type', 'country', 'release_year','listed_in', 'description']
country = ['United States', 'Japan', 'South Korea']

In [200]:
netflix['country'].value_counts()

United States    2555
Name: country, dtype: int64

In [230]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


def Recsys(movie , rank = 10  , country = None , year = 1925, type = None):
    netf = pd.read_csv('netflix_titles.csv')

    feature_cols = ['title','type', 'country', 'release_year','listed_in', 'description']
    netf = netf[feature_cols]

    if type == 'Movie':
        netf = netf[netf['type'] == 'Movie']
    elif type == 'TV Show':
        netf = netf[netf['type'] == 'TV Show']



    # country
    if country != None:
        for i in range(len(country)):
            if i == 0 :
                net = netf[netf['country'] == country[i]]
            else:
                netflix = pd.concat([net, netf[netf['country'] == country[i]]])
    else:
        netflix = netf
    # starting year
    netflix = netflix[netflix['release_year'] >= year]

    word_vec = netflix['description']

    tfidf = TfidfVectorizer(min_df = 2, max_df = 0.7)

    tfidf_vec = tfidf.fit_transform(word_vec)

    # index = title, columns = word, value = word to vector 
    new_netflix = pd.DataFrame(tfidf_vec.toarray(), columns = tfidf.get_feature_names(), index = netflix['title'])

    # drop numeric columns
    string_col = [word for word in list(new_netflix.columns) if word.isalpha() == True]

    new_netflix = new_netflix[string_col]

    result = pd.DataFrame(cosine_similarity(new_netflix, new_netflix), columns = netflix['title'], index = netflix['title'])
    feature_col = list(result.columns)
    cty = netflix['country']
    score = [f'{x * 100 :0.2f} %' for x in result[movie].sort_values(ascending = False)]

    dic = pd.DataFrame(dict({'Movie' : feature_col, 'Similar' : score, 'country' : cty})).reset_index(drop=True)

    return dic.iloc[1:rank,:]

Recsys(movie = 'About Time', type = 'Movie', rank = 10)

,Movie,Similar,country
1,23:59,25.90 %,Singapore
2,9,22.93 %,United States
3,21,22.51 %,United States
4,122,22.09 %,Egypt
5,187,21.95 %,United States
6,706,20.77 %,India
7,1920,20.65 %,India
8,1922,20.63 %,United States
9,"2,215",20.22 %,Thailand


- US와 Japan, South Korea 만 선택해서 추천해줄 것이다.